### Построим модель по данным платформы **KICKSTARTER**, которая будет предсказывать, будет ли выложенный на платформу проект успешным

Импортируем библиотеки и считываем данные

In [127]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/ks.csv')

In [128]:
data.head()

,Название,Категория,Главная категория,Валюта,Дедлайн,Дата публикации,Состояние,Инвесторов,Страна,Собрано в долларах,Цель в долларах
0,"Don't Call it a Comeback ""Telescopes""",Music,Music,USD,2013-01-10,2012-12-09 06:03:52,successful,23,US,600.00,600.00
1,Arcade County (Canceled),Games,Games,USD,2012-04-29,2012-03-30 23:40:45,canceled,5,US,71.00,9000.00
2,Hayashi Skate Co. Solar Skateboard backpack,Accessories,Fashion,CAD,2017-07-22,2017-05-23 23:00:13,canceled,8,CA,360.36,2391.77
3,Me & You Coordinating Sunglasses- Optical Qual...,Accessories,Fashion,USD,2016-11-18,2016-10-19 22:06:41,failed,20,US,502.00,10000.00
4,New Carts for Istanbul Street Food Vendors,Food,Food,USD,2015-05-17,2015-04-17 18:10:47,successful,62,US,2414.00,1400.00


In [129]:
data.shape

(378661, 11)

In [130]:
data['Состояние'].value_counts()

Состояние
failed        197719
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1846
Name: count, dtype: int64

Оставим только те состояния, которые соответствуют законченным проектам, то есть `failed` и `successful`

In [131]:
data = data[data['Состояние'].isin(['failed', 'successful'])]

In [132]:
data.loc[(data['Состояние'] == 'failed'), 'Исход'] = 0
data['Исход'] = data['Исход'].fillna(1)

In [133]:
data = data.drop('Состояние', axis=1)

Будем предсказывать собранное количество денег, применять модель, а потом уже смотреть, нужная ли сумма получилось. Тогда мы решаем задачу регрессии.

In [134]:
### Создадим вещественную колонку таргет
data = data.rename({'Собрано в долларах':'таргет'}, axis=1)

Создадим новый признак *Срок*, который будет показывать разницу в днях между датой публикации и дедлайном сбора средств

In [135]:
### Оставим от колонок с датами разницу в днях 

data['Дедлайн'] = pd.to_datetime(data['Дедлайн'])
data['Дата публикации'] = pd.to_datetime(data['Дата публикации'])

In [136]:
data['Срок'] = (data['Дедлайн'] - data['Дата публикации']).dt.days

В качестве характеристики положения проекта во времени оставим *Год публикации*

In [137]:
### Выделим год запуска проекта
data['Год публикации'] = data['Дата публикации'].dt.year

Добавим котировки на каждую дату в наш датасет

In [138]:
Macro = pd.read_excel("data/macrofeatures.xlsx", engine="openpyxl")

Macro.head()

,Unnamed: 0,Close_brent,Close_sugar,Close_cereals,Close_index_moex,Close_index_moex_10,Close_index_RGBI,Close_index_RTS_oil_and_gas,Close_index_RTS_metallurgy,Close_index_RTS_consumer_sector,Close_index_RTS_telecom,Close_index_RTS_finance,Close_index_RTS_transport,Close_index_RTS_chemicals,Close_index_RTS_broad_market,Close_index_RTS_electricity,dlk_cob_date
0,0,34.41,13.97,442.75,1797.27,3940.81,125.59,123.40,111.97,196.55,70.17,140.57,27.06,177.38,530.59,32.49,2016-02-24
1,1,35.06,14.24,445.25,1803.89,3977.35,126.44,124.22,112.51,198.03,70.56,142.64,27.43,179.48,536.20,33.07,2016-02-25
2,2,35.13,14.00,443.25,1816.73,4027.23,126.90,125.38,113.44,200.13,71.94,145.45,28.06,181.56,544.73,33.55,2016-02-26
3,3,36.64,14.36,445.00,1840.17,4084.24,126.87,126.69,114.66,200.32,72.41,147.22,28.49,186.76,552.82,34.41,2016-02-29
4,4,36.60,14.39,438.50,1844.17,4087.06,127.78,129.72,117.09,204.30,74.26,150.04,30.12,190.67,565.45,34.96,2016-03-01


In [139]:
### Смерджим колонки. Оставим от макрофичей какую-нибудь одну.

Macro = Macro[['Close_brent', 'dlk_cob_date']].drop_duplicates()

In [140]:
Macro['dlk_cob_date'] = pd.to_datetime(Macro['dlk_cob_date'])

In [141]:
data['Дата публикации'] = data['Дата публикации'].dt.date.astype('datetime64[ns]')

In [142]:
data = pd.merge(data,
         Macro,
         left_on=['Дата публикации'],
         right_on=['dlk_cob_date'],
         how='left')

In [143]:
data

,Название,Категория,Главная категория,Валюта,Дедлайн,Дата публикации,Инвесторов,Страна,таргет,Цель в долларах,Исход,Срок,Год публикации,Close_brent,dlk_cob_date
0,"Don't Call it a Comeback ""Telescopes""",Music,Music,USD,2013-01-10,2012-12-09,23,US,600.00,600.00,1.0,31,2012,NaN,NaT
1,Me & You Coordinating Sunglasses- Optical Qual...,Accessories,Fashion,USD,2016-11-18,2016-10-19,20,US,502.00,10000.00,0.0,29,2016,52.54,2016-10-19
2,New Carts for Istanbul Street Food Vendors,Food,Food,USD,2015-05-17,2015-04-17,62,US,2414.00,1400.00,1.0,29,2015,NaN,NaT
3,New Improv Comedy Venue in Des Moines,Theater,Theater,USD,2013-06-17,2013-05-03,86,US,10030.88,10000.00,1.0,44,2013,NaN,NaT
4,The Seer and the Sword,Shorts,Film & Video,USD,2012-08-11,2012-07-12,0,US,0.00,10000.00,0.0,29,2012,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331670,Launch Ladies - A children's book about the Wo...,Children's Books,Publishing,USD,2017-04-20,2017-03-21,555,US,18808.00,9000.00,1.0,29,2017,50.85,2017-03-21
331671,TurnAround//The End Needs A Beginning Album Re...,Rock,Music,GBP,2013-05-15,2013-04-15,0,GB,0.00,2501.45,0.0,29,2013,NaN,NaT
331672,Celtic inspired band looking to move forward w...,Rock,Music,USD,2011-04-29,2011-03-29,0,US,0.00,1500.00,0.0,30,2011,NaN,NaT
331673,Listening & Speaking in Virtual Reality: 5 Int...,Software,Technology,USD,2014-05-09,2014-04-30,1,US,200.00,67000.00,0.0,8,2014,NaN,NaT


Получилось много пропущенных значений, заполним их предыдущими значениями

In [144]:
data = data.sort_values('Дата публикации')

In [145]:
data['Close_brent'] = data['Close_brent'].fillna(method='ffill')

C:\Users\Олег\AppData\Local\Temp\ipykernel_10120\1486187333.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['Close_brent'] = data['Close_brent'].fillna(method='ffill')


Самые первые отсутствующие значения заполним первым неотсутствующим

In [146]:
data['Close_brent'].dropna()

149221    34.41
72755     34.41
235246    34.41
182888    34.41
72631     34.41
          ...  
247730    65.25
318187    66.46
264474    65.99
293634    66.16
23315     66.87
Name: Close_brent, Length: 84850, dtype: float64

In [147]:
# берем 34.41
data['Close_brent'] = data['Close_brent'].fillna(34.41)

Удалим лишние колонки

In [148]:
data = data.drop(['Дедлайн', 'Дата публикации', 'dlk_cob_date'], axis=1)

In [149]:
data.head()

,Название,Категория,Главная категория,Валюта,Инвесторов,Страна,таргет,Цель в долларах,Исход,Срок,Год публикации,Close_brent
176128,Grace Jones Does Not Give A F$#% T-Shirt (limi...,Fashion,Fashion,USD,30,US,625.0,1000.0,0.0,39,2009,34.41
241929,CRYSTAL ANTLERS UNTITLED MOVIE,Shorts,Film & Video,USD,3,US,22.0,80000.0,0.0,87,2009,34.41
244460,drawing for dollars,Illustration,Art,USD,3,US,35.0,20.0,1.0,8,2009,34.41
80845,Offline Wikipedia iPhone app,Software,Technology,USD,25,US,145.0,99.0,1.0,79,2009,34.41
181197,Pantshirts,Fashion,Fashion,USD,10,US,387.0,1900.0,0.0,28,2009,34.41


Удалим также колонки, не являющиеся признаками. 
- *Название* - много уникальных значений, является по сути и индексом, сложно/невозможно установить связь между названием проекта и таргетами
- *Страна* - оставим колонку *Валюта*, которая дублирует информацию
- *Инвесторов* - этот признак формируется одновременно с таргетами, в момент публикации у любого проекта количество инвесторов - 0

In [150]:
data = data.drop(['Название', 'Страна', 'Инвесторов'], axis=1)

In [151]:
data.head()

,Категория,Главная категория,Валюта,таргет,Цель в долларах,Исход,Срок,Год публикации,Close_brent
176128,Fashion,Fashion,USD,625.0,1000.0,0.0,39,2009,34.41
241929,Shorts,Film & Video,USD,22.0,80000.0,0.0,87,2009,34.41
244460,Illustration,Art,USD,35.0,20.0,1.0,8,2009,34.41
80845,Software,Technology,USD,145.0,99.0,1.0,79,2009,34.41
181197,Fashion,Fashion,USD,387.0,1900.0,0.0,28,2009,34.41


Обработаем категориальные признаки. Применим One-hot encoding к колонкам *Валюта* и *Главная категория*


In [152]:
data = pd.concat((data, pd.get_dummies(data['Валюта'])), axis=1)
data = data.drop(['Валюта'], axis=1)

In [153]:
data = pd.concat((data, pd.get_dummies(data['Главная категория'])), axis=1)
data = data.drop(['Главная категория'], axis=1)

In [154]:
### Почистим данные от лишней информации 
data = data.drop('AUD', axis=1)
data = data.drop('Games', axis=1)

In [155]:
data.head()

,Категория,таргет,Цель в долларах,Исход,Срок,Год публикации,Close_brent,CAD,CHF,DKK,...,Design,Fashion,Film & Video,Food,Journalism,Music,Photography,Publishing,Technology,Theater
176128,Fashion,625.0,1000.0,0.0,39,2009,34.41,False,False,False,...,False,True,False,False,False,False,False,False,False,False
241929,Shorts,22.0,80000.0,0.0,87,2009,34.41,False,False,False,...,False,False,True,False,False,False,False,False,False,False
244460,Illustration,35.0,20.0,1.0,8,2009,34.41,False,False,False,...,False,False,False,False,False,False,False,False,False,False
80845,Software,145.0,99.0,1.0,79,2009,34.41,False,False,False,...,False,False,False,False,False,False,False,False,True,False
181197,Fashion,387.0,1900.0,0.0,28,2009,34.41,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [156]:
len(data['Категория'].unique())

159

Признак *Категория* имеет очень много различных значений. Используем mean-target encoding

In [157]:
data['Категория'] = data['Категория'].map(data.groupby(['Категория'])['таргет'].mean())

In [158]:
data.head()

,Категория,таргет,Цель в долларах,Исход,Срок,Год публикации,Close_brent,CAD,CHF,DKK,...,Design,Fashion,Film & Video,Food,Journalism,Music,Photography,Publishing,Technology,Theater
176128,6035.989239,625.0,1000.0,0.0,39,2009,34.41,False,False,False,...,False,True,False,False,False,False,False,False,False,False
241929,3591.033473,22.0,80000.0,0.0,87,2009,34.41,False,False,False,...,False,False,True,False,False,False,False,False,False,False
244460,3661.424550,35.0,20.0,1.0,8,2009,34.41,False,False,False,...,False,False,False,False,False,False,False,False,False,False
80845,4321.245721,145.0,99.0,1.0,79,2009,34.41,False,False,False,...,False,False,False,False,False,False,False,False,True,False
181197,6035.989239,387.0,1900.0,0.0,28,2009,34.41,False,False,False,...,False,True,False,False,False,False,False,False,False,False


Разделим данные на объекты и ответы

In [159]:
X = data.drop('таргет', axis=1)
Y = data['таргет']

Обучим модель

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, Y)

X['Предсказание'] = model.predict(X)

In [163]:
X.head()

,Категория,Цель в долларах,Исход,Срок,Год публикации,Close_brent,CAD,CHF,DKK,EUR,...,Fashion,Film & Video,Food,Journalism,Music,Photography,Publishing,Technology,Theater,Предсказание
176128,6035.989239,1000.0,0.0,39,2009,34.41,False,False,False,False,...,True,False,False,False,False,False,False,False,False,-5242.382987
241929,3591.033473,80000.0,0.0,87,2009,34.41,False,False,False,False,...,False,True,False,False,False,False,False,False,False,-248.270446
244460,3661.424550,20.0,1.0,8,2009,34.41,False,False,False,False,...,False,False,False,False,False,False,False,False,False,5285.539639
80845,4321.245721,99.0,1.0,79,2009,34.41,False,False,False,False,...,False,False,False,False,False,False,False,True,False,26503.212042
181197,6035.989239,1900.0,0.0,28,2009,34.41,False,False,False,False,...,True,False,False,False,False,False,False,False,False,-7399.263126


In [164]:
data.head()

,Категория,таргет,Цель в долларах,Исход,Срок,Год публикации,Close_brent,CAD,CHF,DKK,...,Design,Fashion,Film & Video,Food,Journalism,Music,Photography,Publishing,Technology,Theater
176128,6035.989239,625.0,1000.0,0.0,39,2009,34.41,False,False,False,...,False,True,False,False,False,False,False,False,False,False
241929,3591.033473,22.0,80000.0,0.0,87,2009,34.41,False,False,False,...,False,False,True,False,False,False,False,False,False,False
244460,3661.424550,35.0,20.0,1.0,8,2009,34.41,False,False,False,...,False,False,False,False,False,False,False,False,False,False
80845,4321.245721,145.0,99.0,1.0,79,2009,34.41,False,False,False,...,False,False,False,False,False,False,False,False,True,False
181197,6035.989239,387.0,1900.0,0.0,28,2009,34.41,False,False,False,...,False,True,False,False,False,False,False,False,False,False
